In [1]:
"""
Created on Sun Mar  6 15:55:40 2022

@author: 99259
"""
from sklearn.metrics import confusion_matrix
import package_dataprocess.Nilm_classes
import numpy as np
import os
import gc
import pandas as pd
from PIL import Image
import PIL.ImageOps
import matplotlib.pyplot as plt

一切都是命运石之门的选择！


In [2]:
fridge_data = np.zeros(100800)
television_data = np.zeros(100800)
kettle_data = np.zeros(100800)
microwave_data = np.zeros(100800)
washerdryer_data = np.zeros(100800)
sum_data_num = np.zeros(100800)

In [3]:
#只要在这里改周数和每一周开始的unix时间戳即可
week = 'week3'
unixtime = 1453075200

In [4]:

filenames = os.listdir(r'DATA\\UKData_by_hour\\'+week+'\\LowFreq')
dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA\\UKData_by_hour\\"+week+"\\vicurve_name.xlsx",
                                                       path_img="DATA\\UKData_by_hour\\"+week+"\\vicurve32x32\\",
                                                       dimension = 5)

for time in range(7 * 24):
    temp = np.zeros(100800)
    for i in filenames:
        if (i[:10] == str(unixtime + 3600 * time)):#换周的话记得换一下时间戳喔
            name = i[11:-4]
            name_data = np.load('DATA\\UKData_by_hour\\'+week+'\\LowFreq\\' + i)
            name_data = np.array(name_data)
            # print(name_data.shape)
            if len(name_data) > 600:
                name_data = name_data[:600]
            elif len(name_data) < 600:
                name_data = np.pad(name_data, (0, 600 - len(name_data)), 'constant', constant_values=(0, 0))
            name_data = np.where(name_data > 3, name_data, 0)  # 底噪3w

            if (name == 'Fridge freezer0'):
                fridge_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Television0'):
                television_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Kettle0'):
                kettle_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Microwave0'):
                microwave_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Washer dryer0'):
                washerdryer_data[time * 600:(time + 1) * 600] = name_data

            temp[time * 600:(time + 1) * 600] = name_data
            sum_data_num = sum_data_num + temp

fridge_data = fridge_data.reshape(fridge_data.shape[0], 1)
television_data = television_data.reshape(television_data.shape[0], 1)
kettle_data = kettle_data.reshape(kettle_data.shape[0], 1)
microwave_data = microwave_data.reshape(microwave_data.shape[0], 1)
washerdryer_data = washerdryer_data.reshape(washerdryer_data.shape[0], 1)

In [6]:
#创建标签
fridge_labels = dp.create_label(fridge_data, 0, 15)#15表示有功功率阈值
kettle_labels = dp.create_label(kettle_data, 1, 15)
microwave_labels = dp.create_label(microwave_data, 2, 15)
tv_labels = dp.create_label(television_data, 3, 15)
washerdryer_labels = dp.create_label(washerdryer_data, 4, 15)

sum_label = fridge_labels + tv_labels + kettle_labels + microwave_labels + washerdryer_labels
#print(sum_label.shape)
sumlabel = sum_label.T

In [7]:
#sum_data标准化
mean = sum_data_num[:100800].mean(axis=0)
sum_data = sum_data_num - mean
std = sum_data[:100800].std(axis=0)
sum_data /= std
#sum_data从list转化为numpy数组便于后续处理
sum_data = np.array(sum_data)
#print(sum_data.shape)
#sum_data从list转化为numpy数组便于后续处理
sum_data = sum_data.reshape(sum_data.shape[0], 1)
print(sum_data.shape)

(100800, 1)


In [4]:
#计数变量，统计标签为1的个数
fridge = 0
kettle = 0
microwave = 0
tv = 0
washerdryer = 0
for i in range(100800):
    if(sumlabel[0][i]==1):
        fridge+=1
    if(sumlabel[1][i]==1):
        kettle+=1
    if(sumlabel[2][i]==1):
        microwave+=1
    if(sumlabel[3][i]==1):
        tv+=1
    if(sumlabel[4][i]==1):
        washerdryer+=1

In [5]:
print(fridge/100800)
print(kettle/100800)
print(microwave/100800)
print(tv/100800)
print(washerdryer/100800)

0.36989087301587303
0.00744047619047619
0.010793650793650795
0.15930555555555556
0.027043650793650793
